DBMS 처리 과정
## SQLite3 실습

In [5]:
import sqlite3  # SQLite에서는 Boolean, VarChar을 따로 지원하지 않는다.

DBMS
->Connection (Connection Pool) #
->Cursor (Client - Server)
        SQL ->
                Result
        Cursor ->
        Fetch

conn = sqlite3.connect('경로/이름/' or 'memory:')

In [6]:
con = sqlite3.connect(':memory:') 
cur = con.cursor()

In [7]:
type(con), type(cur)

(sqlite3.Connection, sqlite3.Cursor)

In [8]:
type(con.cursor())  # con에도 cursor 객체가 있다. cursor에 붙어있는 객체 반환

sqlite3.Cursor

execute : 비공식적 방법. 기본적으로 단발성이기 때문에 세미콜론(;)으로 마무리하지 않아도 된다.

In [9]:
cur.execute('CREATE TABLE a (id, name);') 

In [10]:
cur.execute('')

In [11]:
cur.execute('INSERT INTO a(id, name) VALUES(1, "name");')

In [12]:
cur.execute('SELECT * FROM a;')  # 다시 할 때는 여기서부터 시작해도 됨.

In [13]:
#[_ for _ in cur]  # 비공식적 방법

In [14]:
cur.fetchall()  # 공식적 방법

[(1, 'name')]

In [15]:
cur.execute('INSERT INTO a (id, name) VALUES (?, ?);', [2,'name2'])

In [16]:
cur.execute('INSERT INTO a (id, name) VALUES (?, ?);', [3,'name3'])

In [17]:
cur.fetchall()  # 공식적 방법

[]

In [18]:
cur.execute('INSERT INTO a (id, name) VALUES (:id, :name);', {'id':3 ,'name':'name3'})

In [19]:
data = [(_+3, 'name'+str(_+3)) for _ in range(5)]

In [20]:
cur.executemany('INSERT INTO a(id, name) VALUES(?, ?);', data)

In [21]:
cur.execute('SELECT * FROM a;')  # 다시 할 때는 여기서부터

In [22]:
cur.fetchall()  # 공식적 방법

[(1, 'name'),
 (2, 'name2'),
 (3, 'name3'),
 (3, 'name3'),
 (3, 'name3'),
 (4, 'name4'),
 (5, 'name5'),
 (6, 'name6'),
 (7, 'name7')]

In [23]:
for _ in con.iterdump():  # history 볼 수 있음.
    print(_)

BEGIN TRANSACTION;
CREATE TABLE a (id, name);
INSERT INTO "a" VALUES(1,'name');
INSERT INTO "a" VALUES(2,'name2');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(4,'name4');
INSERT INTO "a" VALUES(5,'name5');
INSERT INTO "a" VALUES(6,'name6');
INSERT INTO "a" VALUES(7,'name7');
COMMIT;


데이터베이스 적재하기

In [24]:
con = sqlite3.connect('temp1.db')
cur = con.cursor()

In [25]:
cur.executescript('''
BEGIN TRANSACTION;
CREATE TABLE a (id, name);
INSERT INTO "a" VALUES(1,'name');
INSERT INTO "a" VALUES(2,'name2');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(3,'name3');
INSERT INTO "a" VALUES(4,'name4');
INSERT INTO "a" VALUES(5,'name5');
INSERT INTO "a" VALUES(6,'name6');
INSERT INTO "a" VALUES(7,'name7');
COMMIT;

''')

In [26]:
cur.execute('SELECT * FROM a')
cur.fetchall()

[(1, 'name'),
 (2, 'name2'),
 (3, 'name3'),
 (3, 'name3'),
 (3, 'name3'),
 (4, 'name4'),
 (5, 'name5'),
 (6, 'name6'),
 (7, 'name7')]

In [27]:
cur.execute('INSERT INTO a VALUES(999, "TEST")')  # 여기까지 하면 메모리에 들어는 있으나 물리적인 DB에는 들어있지 않다. -> commit 필요

In [28]:
cur.execute('SELECT * FROM a')  # 
cur.fetchall()

[(1, 'name'),
 (2, 'name2'),
 (3, 'name3'),
 (3, 'name3'),
 (3, 'name3'),
 (4, 'name4'),
 (5, 'name5'),
 (6, 'name6'),
 (7, 'name7'),
 (999, 'TEST')]

In [29]:
con.commit()  # 명시적으로 transaction이 반응하도록

excutescript : 세미콜론으로 구분된 연속된 SQL문장을 수행

In [30]:
con.close()  # 정말 최종에만 사용

## 음악 재생 목록 만들기

노래는 앨범에 소속. 동일한 노래는 서로 다른 앨범에 등장 가능. 특정 가수에게는 여러 곡의 노래가 있을 수 있고, 동일한 제목이여도 서로 다른 가수가 불렀을 수 있다. 

In [32]:
con = sqlite3.connect('playlist.db')
cur = con.cursor()

In [34]:
cur.execute('''
        CREATE TABLE artist (
            pk INTEGER PRIMARY KEY AUTOINCREMENT, 
            name TEXT NOT NULL
        
        )
''')

In [39]:
cur.execute('''
        CREATE TABLE album (
            pk INTEGER PRIMARY KEY AUTOINCREMENT, 
            name TEXT NOT NULL
        
        )
''')

In [40]:
cur.execute('''
        CREATE TABLE genre (
            pk INTEGER PRIMARY KEY AUTOINCREMENT, 
            name TEXT NOT NULL
        
        )
''')

In [38]:
cur.execute('''
        CREATE TABLE track (
            pk INTEGER PRIMARY KEY AUTOINCREMENT, 
            name TEXT NOT NULL,
            length TEXT NOT NULL,
            rating INTEGER NOT NULL,
            count INTEGER NOT NULL,
            album_fk INTEGER NOT NULL,
            genre_fk INTEGER NOT NULL
        )
''')

'가수', '앨범', '노래', '장르', '길이', '0'(rating), '0'(count)

In [42]:
temp = [('가수1',), ('가수2',), ('가수3',), ('가수4',), ('가수5',)]
cur.executemany('INSERT INTO artist(name) VALUES(?)', temp)

In [43]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4'), (5, '가수5')]

In [44]:
temp = [('발라드',), ('KPOP',), ('힙합',), ('인디',), ('댄스',)]
cur.executemany('INSERT INTO genre(name) VALUES(?)', temp)

In [45]:
# 관
cur.execute('''
    ALTER TABLE album
    ADD COLUMN artist_fk INTEGER NOT NULL
''')

In [46]:
con.commit()

1. 아티스트로부터 pk값을 찾아낸다.
2. SQL로 해결

In [47]:
# 1번 방법 : Database -> Local (변수확인), 완벽일치 해야함.
cur.execute('SELECT * FROM artist ORDER BY pk ASC')
T_ARTIST = cur.fetchall()

In [49]:
for _ in T_ARTIST:
    if _[1] == '가수1':  # 중복된 데이터가 들어갔을 가능성이 높다.
        print(_[0])
        

1


In [48]:
sqlite3.version, sqlite3.sqlite_version

('2.6.0', '3.36.0')

In [50]:
# 2번 방법 : 완벽일치하지 않아도 된다는 장점.
cur.execute('SELECT pk FROM artist WHERE name LIKE "가수1"')
cur.fetchall()

[(1,)]

In [51]:
cur.execute('SELECT pk FROM artist WHERE name LIKE "%1"') 
cur.fetchall()

[(1,)]

In [56]:
cur.executemany('''
    INSERT INTO album (name, artist_fk)
    VALUES ('앨범1',
    (SELECT pk FROM artist
    WHERE name LIKE "%가수%"
    ORDER BY pk ASC
    LIMIT 0,1))
''')   
#cur.fetchall()
# %?%을 사용하면 [('가',)]와 같은 조건문을 달 수도 있음.

TypeError: function takes exactly 2 arguments (1 given)

In [69]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, '앨범1', 5),
 (2, '앨범1', 2),
 (3, '앨범1', 1),
 (4, '앨범1', 1),
 (5, '앨범1', 2),
 (6, '앨범1', 5),
 (7, '앨범1', 5),
 (8, '앨범1', 2),
 (9, '앨범1', 3),
 (10, '앨범1', 4)]

In [ ]:
cur.execute('''
    INSERT INTO album (name, artist_fk)
    VALUES ('앨범1'
    (SELECT pk FROM artist
    WHERE name LIKE "%가수%"
    ORDER BY pk ASC
    LIMIT 0,1))
''')  

In [62]:
from random import choice
cur.execute('SELECT name FROM artist')
artist = cur.fetchall()
choice(artist)

for _ in range(10):
    cur.execute('''
        INSERT INTO album (name, artist_fk)
        VALUES ('앨범1',
        (SELECT pk FROM artist
        WHERE name=?
        ORDER BY pk ASC
        LIMIT 0,1))
    ''', choice(artist))

In [72]:
cur.execute('''
    SELECT B.pk, A.name, B.name, C.name, D.name
    FROM artist as A
    INNER JOIN album AS B on B.artist_fk = A.pk
    INNER JOIN track AS C on C.album_fk = B.pk
    INNER JOIN genre AS D on C.genre_fk = D.pk
''')
cur.fetchall()

[(4, '가수1', '앨범1', '노래4', '인디'),
 (4, '가수1', '앨범1', '노래4', '인디'),
 (5, '가수2', '앨범1', '노래5', '댄스'),
 (1, '가수5', '앨범1', '노래1', '발라드'),
 (5, '가수2', '앨범1', '노래5', '댄스'),
 (1, '가수5', '앨범1', '노래1', '발라드'),
 (2, '가수2', '앨범1', '노래2', 'KPOP'),
 (4, '가수1', '앨범1', '노래4', '인디'),
 (5, '가수2', '앨범1', '노래5', '댄스'),
 (5, '가수2', '앨범1', '노래5', '댄스')]

In [64]:
track = ['노래1', '노래2', '노래3', '노래4', '노래5', '노래6']
genre = ['발라드', 'KPOP', '힙합', '인디', '댄스']
album = [1,2,3,4,5,6,7]



In [66]:
for _ in range(10):
    i = choice(range(5))
    cur.execute('''
        INSERT INTO track (name, length, rating, count, album_fk, genre_fk)
        VALUES(
        ?, 0, 0, 0,(
            SELECT pk FROM album WHERE pk=? LIMIT 0,1
            ),(
            SELECT pk FROM genre WHERE name=?
            ))

    ''', [track[i], album[i], genre[i]])

LIMIT 0,1 : 유무를 확인하는 query

In [67]:
con.commit()

In [68]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래4', '0', 0, 0, 4, 4),
 (2, '노래4', '0', 0, 0, 4, 4),
 (3, '노래5', '0', 0, 0, 5, 5),
 (4, '노래1', '0', 0, 0, 1, 1),
 (5, '노래5', '0', 0, 0, 5, 5),
 (6, '노래1', '0', 0, 0, 1, 1),
 (7, '노래2', '0', 0, 0, 2, 2),
 (8, '노래4', '0', 0, 0, 4, 4),
 (9, '노래5', '0', 0, 0, 5, 5),
 (10, '노래5', '0', 0, 0, 5, 5)]

ORM : SQLAlchemy 사용
객체 기반의 데이터베이스 생성을 할 수 있음
- 활용 : 정보 조회가 다수 필요한 분야